In [ ]:
!pip3 install pandas
!pip3 install numpy
!pip3 install python-dotenv
!pip3 install sqlalchemy
!pip3 install psycopg2-binary

In [1]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from os import getenv, listdir
import csv
from sqlalchemy import create_engine
import psycopg2
from monitors_format_settings import monitor_settings
from datetime import timedelta

In [2]:
# Extract monitor settings
monitor_settings = monitor_settings

In [3]:
# Extract environment variables
load_dotenv()
DBNAME=getenv('DBNAME')
DBUSER=getenv('DBUSER')
DBHOST=getenv('DBHOST')
DBPORT=getenv('DBPORT')
DBPWRD=getenv('DBPWRD')

# Connect to database

## Via sqlalchemy for sending tables

In [91]:
try:
    engine = create_engine(
        url="postgresql://{0}:{1}@{2}:{3}/{4}".format(
            DBUSER, DBPWRD, DBHOST, DBPORT, DBNAME
        )
    )
    print('Connected to muon database successfully')
except:
    print('Unable to connect to muon database')

Connected to muon database successfully


## Via psycopg2 to send queries

In [92]:
try:
    conn = psycopg2.connect(f"dbname={DBNAME} user={DBUSER} host={DBHOST} port={DBPORT} password={DBPWRD}")
    print('Connected to muon database successfully')
except:
    print('Unable to connect to muon database')
cur = conn.cursor()

Connected to muon database successfully


# Create Monitor table with locations

In [ ]:
monitor_locations = pd.read_csv('../data/monitor_locations.csv').set_index('name')
monitor_locations.to_sql(
    con=engine, name='monitors', if_exists='replace', index=True)
print('Table sent to db successfully')

In [ ]:
# Create primary key for monitors table
cur.execute("""ALTER TABLE monitors ADD PRIMARY KEY (name)""")
conn.commit()
cur.close()
conn.close()

# Create functions for data formatting and standarization

## Formats each log as a date and counts df, based on each unique monitor's data format

In [15]:
def log_df_formatting(logs_df, monitor_name):
    ''' 
    Formats given logs into a datetime and counts column df. If desired, you can keep the
    additional counts columns in case needed for later.
    
    Args:       logs_df         -> pandas df containing non-formatted data where only one column
                                    named 'counts' exists and has all data.
                monitor_name    -> indicates how to split the row based on specific monitor's
                                    row content, since some have more data than others
    
    Returns:    pandas df containing formatted logs as date index and counts column

    '''
    monitors = monitor_settings
    # Get specific split number of elements depending monitor log file data format
    splits = monitors[monitor_name]['splits']
    # Get counts column
    counts_col = monitors[monitor_name]['counts_col']
    # Get index for date column
    date_col = monitors[monitor_name]['date_col']
    # Get corresponding timezone
    monitor_tz = monitors[monitor_name]['timezone']

    # Make copy to ensure it is not damaging current logs data
    temp = logs_df.copy(deep=True)

    # Format df to remove tracing commas after counts values if the separation is based on spacing
    if temp['counts'].str.contains(' ').any() & temp['counts'].str.contains(',').any():
        temp['counts'] = temp['counts'].str.replace(',','', regex=True)
        
    # Split counts column into n elements (i.e.: if n=4, containing count1, count2, count3, day)
    # separation is space-based " "
    if temp['counts'].str.contains(' ').any():
        temp = temp['counts'].str.split(" ", n = splits, expand=True)
    # else, separation is comma-based ","
    else:
        temp = temp['counts'].str.split(",", n=splits, expand=True)
    print(temp.head(1))
    # Slice df to only have desired columns counts and date
    temp = temp.rename(columns={date_col:'date',counts_col:'counts'})
    ldf = temp[['date', 'counts']]

    # transform date column string into a datetime format type
    print('First date for formatting: ', ldf.iloc[0]['date'])
    ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
    # Localize to monitor location's timezone, ambiguous to infer fall DST change an hour back, 
    # and nonexistent for clocks moving forward due to DST
    print('date check prior to localize: ', ldf.head(1))

    try:
        # It infers the date and shifts forward if dailight savings causes errors
        ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
    except:
        try:
            # If duplicate dates due to localization into a timezone with daylight, choose to keep one
            ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous=True, nonexistent='shift_forward')
        except:
            # Else, the df is already localized to a particular timezone
            print('DF date already timezone aware so no need!')

    # Format to UTC so it can be merged with the weather app's data that is also UTC
    ldf['date'] = ldf['date'].dt.tz_convert('UTC')
    # Set date column as index
    ldf = ldf.set_index('date')
    # Make counts column into numeric type
    ldf['counts'] = ldf['counts'].apply(pd.to_numeric)

    # Remove any errors due to NaT values on index date column
    try:
        ldf2 = ldf.drop('NaT')
    except:
        ldf2 = ldf

    # Sort df based on date index
    ldf2.sort_index()
    
    return ldf2

## Makes any found log files within monitor folder into a pandas df and stores as a list

In [7]:
def all_monitor_logs_to_dfs(monitor_name):
    ''' 
    Creates a df of merged log files. This assumes all logs within given monitor folder
    name share same format style. Log files should be reviewed before applying this function.

    Args:       monitor_name -> name of folder containing log files to be merged
    Returns:    merged pandas df

    '''
    # Home directory
    homedir = '../data/'
    # list of files within selected monitor
    all_files = listdir(homedir + monitor_name)
    # filter based on ending being .log
    logs_list = [f for f in all_files if f.endswith('.log')]

    # df = pd.DataFrame()
    logs_df_list = []

    for log_file in logs_list:
        # Read log file as df separating rows
        temp = pd.read_csv(homedir+monitor_name+'/'+log_file, sep='\t', names=['counts'])
        # If df has column titles, remove as it causes issues with below functions
        if temp.head(1)['counts'].str.contains('date').any():
            temp.drop(temp.head(1).index, inplace=True)
        # Format df based on monitor settings
        temp = log_df_formatting(temp, monitor_name.replace('/',''))
        # Append to list of dfs
        logs_df_list.append(temp)

    return logs_df_list

## merge logs with shutdown - adds time interval between monitor offline and online times

In [80]:
def merge_adding_all_timestamps_hourly(merged_logs, log_df, lfreq, trim_base_df, trim_log_df):

    # Format to hourly and trim if requested
    if not merged_logs.empty and trim_base_df:
        temp = merged_logs.resample('h').sum()
        temp.drop(temp.head(1).index, inplace=True)
        temp.drop(temp.tail(1).index, inplace=True)
        temp.loc[temp['counts'] == 0] = np.nan
        temp = temp[temp.first_valid_index():]
        merged_logs = temp

    if not log_df.empty and trim_log_df:
        temp = log_df.resample('h').sum()
        temp.drop(temp.head(1).index, inplace=True)
        temp.drop(temp.tail(1).index, inplace=True)
        temp.loc[temp['counts'] == 0] = np.nan
        temp = temp[temp.first_valid_index():]
        log_df = temp

    # Check if df is not empty after trimming and perform merge
    if not merged_logs.empty and not log_df.empty:

        # Extract first date from log df to be merged at end of current base df
        first_log_date = str(log_df.head(1).index[0])
        # Extract last date from base df
        last_log_date = str(merged_logs.tail(1).index[0])

        # Check if dates are equal or in the right chronological order ascending
        if str(merged_logs.head(1).index[0]) <= first_log_date:
            # If able to find first log date from log df on merged_logs, no detector 
            # has shutdown and just need to add at end of file whatever is not yet part of base df
            try:
                bf_first_log_date = str(merged_logs.iloc[merged_logs.get_loc(first_log_date) - 1].name)
                print('No shutdown since last log data integration')
                # Merge
                merged = pd.concat([merged_logs, log_df])
                final_merged_logs = merged[~merged.index.duplicated(keep='last')]
                # Sort fixed merged log files
                merged = final_merged_logs.sort_index()
            
            # else, logs must be merged considering missing counts due to detector shutoff time
            except:
                print('Last log date bf shutdown: ', last_log_date, '\nFirst log date after shutdown: ', first_log_date)
                # Create continuous range of dates to complete df timeline for missing date indexes
                date_range = pd.date_range(start=last_log_date, end=first_log_date, freq=lfreq, tz='UTC')
                # Create df containing missing dates as index and np.nan values
                offline_df = pd.DataFrame(index=date_range, columns=merged_logs.columns, data=np.nan)
                # Update current logs to include missing detector data time stamps but with nan values for graph
                complete_logs = pd.concat([offline_df, log_df])
                merged = pd.concat([merged_logs, complete_logs])
                final_merged_logs = merged[~merged.index.duplicated(keep='last')]
                # Sort fixed merged log files
                merged = final_merged_logs.sort_index()

        # Reverse which file is merged at end of the other to ensure chronological order
        else:
            # Extract first date from log df to be merged at end of current base df
            first_log_date = str(merged_logs.head(1).index[0])
            # Extract last date from base df
            last_log_date = str(log_df.tail(1).index[0])

            try:
                bf_first_log_date = str(log_df.iloc[log_df.get_loc(first_log_date) - 1].name)
                print('No shutdown since last log data integration')
                # Merge
                merged = pd.concat([log_df, merged_logs])
                final_merged_logs = merged[~merged.index.duplicated(keep='last')]
                # Sort fixed merged log files
                merged = final_merged_logs.sort_index()
            
            # else, logs must be merged considering missing counts due to detector shutoff time
            except:
                print('Last log date bf shutdown: ', last_log_date, '\nFirst log date after shutdown: ', first_log_date)
                # Create continuous range of dates to complete df timeline for missing date indexes
                date_range = pd.date_range(start=last_log_date, end=first_log_date, freq=lfreq, tz='UTC')
                # Create df containing missing dates as index and np.nan values
                offline_df = pd.DataFrame(index=date_range, columns=merged_logs.columns, data=np.nan)
                # Update current logs to include missing detector data time stamps but with nan values for graph
                complete_logs = pd.concat([offline_df, merged_logs])
                merged = pd.concat([log_df, complete_logs])
                final_merged_logs = merged[~merged.index.duplicated(keep='last')]
                # Sort fixed merged log files
                merged = final_merged_logs.sort_index()
    
    # Only merged logs remains after trimming
    elif not merged_logs.empty:
        merged = merged_logs
    # Only log df remains after trimming
    elif not log_df.empty:
        merged = log_df
    # None have data, return empty df
    else:
        merged = pd.DataFrame()

    return merged

## Merge log dfs - merges list of logs together, and if provided, merges that list to a csv file

In [9]:
def merge_log_dfs(monitor_name, log_dfs_list, logs_csv=None):
    ''' 
    Organizes each log df within a list of dfs chronologically, then merges them all.
    This includes adding missing time between end of one log to the other as np.NaN
    values due to monitor shutdowns.

    Args:       log_dfs_list -> list containing n number of log dfs from a specific
                                monitor data folder
                logs_csv    ->  Variable in case it is a log file being merged to a csv df or file
                                Originally set to None
    Returns:    merged pandas df with all logs

    '''
    if len(log_dfs_list) == 0 or log_dfs_list == None:
        print('Given list of log dfs is empty. Try again with to run the all_monitor_logs_to_dfs() with correct file path')
    elif len(log_dfs_list) > 1:
        
        # Sort logs from latest to earliest based on initial log date to ensure optimal merging
        # Extract first row date of each log df into list
        logs_l = [str(df.head(1).index[0]) for df in log_dfs_list]
        # Make dictionary containing index of corresponding date on provided logs list
        logs_dict = {}
        for i in range(len(logs_l)):
            logs_dict[logs_l[i]] = i
        # sort values in ascending order
        logs_l.sort()
        # Create sorted list of logs
        new_logs_df_list = []
        for i in range(len(logs_l)):
            new_logs_df_list.append(log_dfs_list[logs_dict[logs_l[i]]])
        
        # Extract first df on list and merged with next, then repeat
        merged_logs = new_logs_df_list[0].copy(deep=True)
        # Extract frequency of data from given monitor
        lfreq = monitor_settings[monitor_name]['freq']
        # Flag for initial merged df to be trimmed for incomplete hourly logs
        trim_merged = True

        # Navigate through list of dfs and merge
        for i in range(1, len(new_logs_df_list)):
            print(f'Log file index {i}')
            merged_logs = merge_adding_all_timestamps_hourly(merged_logs, new_logs_df_list[i], lfreq, trim_merged, True)
            trim_merged = False # Set flag to false to no longer trim base df as it has been done

    # Else, the df list only has one df so no merging within list
    else: 
        merged_logs = new_logs_df_list[0].copy(deep=True)
        # Sample by hour
        merged_logs1 = merged_logs.resample('h').sum()
        # Trim first and last hour because of data missing due to log being incomplete
        merged_logs1.drop(merged_logs1.head(1).index, inplace=True)
        merged_logs1.drop(merged_logs1.tail(1).index, inplace=True)
        # Reassign for return statement accuracy
        merged_logs = merged_logs1
        
    # If a csv filepath or csv df has been provided for merging with bulk merging of any logs within log list
    if logs_csv != None:
        # If csv file is given as a name path string
        if type(logs_csv) == str and logs_csv[-3:] == 'csv':
            # CSV file into a dataframe and format to have datetime and numeric columns
            df1 = pd.read_csv(logs_csv, names=['date','counts'])[1:] # Remove extra first row
            df1["date"] = pd.to_datetime(df1['date'])
            df1['date'] = df1['date'].dt.tz_convert('UTC') # convert time zone to UTC
            df1 = df1.set_index('date')
            df1["counts"] = df1["counts"].apply(pd.to_numeric)
        # Else, it is a pandas df
        elif type(logs_csv) == pd.core.frame.DataFrame:
            df1 = logs_csv.copy(deep=True)

        # Else, file type is not a csv nor df
        else:
            print('Please provide a file with csv or dataframe format for the logs_csv variable!')

        # Merge the merged logs dfs into the csv df
        merged_logs = merge_adding_all_timestamps_hourly(df1, merged_logs, lfreq, True, False)

    return merged_logs

## Formats csv containing only dates and counts, any other format will fail

In [10]:
def format_csv(csv_path):
    # CSV file into a dataframe and format to have datetime and numeric columns
    df1 = pd.read_csv(csv_path, names=['date','counts'])[1:] # Remove extra first row
    df1["date"] = pd.to_datetime(df1['date'])
    df1['date'] = df1['date'].dt.tz_convert('UTC') # convert time zone to UTC
    df1 = df1.set_index('date')
    df1["counts"] = df1["counts"].apply(pd.to_numeric)
    return df1

## Creates csv and sends it to DB

In [11]:
def process_and_save_logs(monitor_name, csv_path=None):

    print(f'Processing any logs for monitor {monitor_name}')
    # Get list of all monitor logs into a df
    log_dfs_list = all_monitor_logs_to_dfs(monitor_name)
    # Merge all found logs to df
    hourly_logs = merge_log_dfs(monitor_name.replace('/',''), log_dfs_list)

    if csv_path != None:
        hourly_logs = merge_log_dfs(monitor_name.replace('/',''), hourly_logs, logs_csv=csv_path)

    # Create CSV
    new_csv=f'{monitor_name.replace('/','_')}_all_logs.csv'
    hourly_logs.to_csv(f'../data/{monitor_name}/{new_csv}')

    # Check formatting of name doesn't have number up front
    monitor_name = monitor_name.replace('/','').lower()
    if monitor_name.startswith('2') or monitor_name.startswith('4'):
        monitor_name = monitor_name[1:]+monitor_name[0]

    # Assumes DB connections have been established
    # Send to DB via SQLALCHEMY
    hourly_logs.to_sql(
        con=engine, name=f'{monitor_name}', if_exists='replace', index_label='date')
    print('Table sent to DB successfully')

    # Make primary key for table via PSYCOPG2
    cur = conn.cursor()
    cur.execute(f"""ALTER TABLE {monitor_name} ADD PRIMARY KEY (date)""")
    conn.commit()
    cur.close()
    print('Query for primary key sent successfully')

## What to do with this?

In [ ]:
def log_to_df(log_file):
    ''' 
    Transforms a given log file to a pandas dataframe already formatted with columns date and counts, 
    and only follows format of paddle2, paddle4, and markv log files

    Args:       log_file -> name of log file including path location
    Returns:    pandas df

    '''

    # Create new df with downloaded logs file, example name -> "./logs/4Paddle_2024-08-07_to_2024-08-16.log"
    df = pd.read_csv(log_file, sep='\t', names=['counts'])
    # Split data by spacing, then into a group of 3 elements (i.e., column 1: 385 column 2: 0 column 3:0 column 4: Wed Oct 11 17:17:30 2023)
    df = df['counts'].str.split(" ", n=3, expand=True)
    # Create df and indicate column names for new columns
    df2 = pd.DataFrame(df.values, columns=["counts", "junk1", "junk2", "date"])
    
    # Format date column to date value
    df2["date"] = pd.to_datetime(df2['date'], format="%a %b %d %H:%M:%S %Y")
    # Transform date into current timezone
    df2['date'] = df2['date'].dt.tz_localize('America/New_York', ambiguous='infer')
    # convert time zone to UTC (Cordinated Universal Time)
    df2['date'] = df2['date'].dt.tz_convert('UTC')

    # make date column the index
    df2 = df2.set_index('date')

    # convert counts column data to numeric format -> in case strings appear, it won't cause errors in contrast with astype(int)
    df2["counts"] = df2["counts"].apply(pd.to_numeric)

    # clean df by removing columns junk1 and junk2
    df2.drop(df2.columns[[1, 2]], axis=1, inplace=True)
    
    # Remove any errors due to NaT values on index date column
    try:
        df2_c = df2.drop('NaT')
    except:
        df2_c = df2
    print('Log file first and last records bf trimming: \n', df2_c.head(1), '\n', df2_c.tail(1))

    # resampling hourly and getting sum due to logs being recorded for every minute
    df2_hourly = df2_c.resample('h').sum()
    
    # Drop first and last hour on log file due to hourly data count giving innacurate total
    # resulting from starting and ending halfway through the hour/incomplete count
    df2_hourly.drop(df2_hourly.head(1).index, inplace=True)
    df2_hourly.drop(df2_hourly.tail(1).index, inplace=True)
    print('Log file first and last records after trimming: \n', df2_hourly.head(1), '\n', df2_hourly.tail(1))

    return df2_hourly

In [ ]:
def merge_csv_with_logs(logs_csv, new_logs):
    ''' 
    Merges any available csv with a given log file of a specific format and naming convention like 
    2Paddle, 4Paddle, and MarkV log files.
    
    Args:       logs_csv -> csv file name containing all merged logs since last execution of function
                new_logs -> log file name downloaded from a detector server by the cosmic server, then
                        downloaded to the database server for this web app
                new_csv_name -> string for new file's name
            
    Returns:   merged_logs -> csv file with all merged logs as df, then also stores new csv version
    
    * DISCLAIMER: It assumes the paths for the csv file is fixed and no need
    to be directly provided. If administrator changes path route, he must correctly assign new path

    '''
    # If file is given as a name path string
    if type(logs_csv) == str and logs_csv[-3:] == 'csv':
        # CSV file into a dataframe and format to have datetime and numeric columns
        df1 = pd.read_csv(logs_csv, names=['date','counts'])[1:] # Remove extra first row
        df1["date"] = pd.to_datetime(df1['date'])
        df1['date'] = df1['date'].dt.tz_convert('UTC') # convert time zone to UTC
        df1 = df1.set_index('date')
        df1["counts"] = df1["counts"].apply(pd.to_numeric)
    
    # Else, it is a log file never formatted before
    elif type(logs_csv) == str and logs_csv[-3:] in ['log','dat']:
        df1 = log_to_df(logs_csv)
        
    # Else, it is a pandas df
    elif type(logs_csv) == pd.core.frame.DataFrame:
        df1 = logs_csv.copy(deep=True)

    # Else, file type is not a csv nor df
    else:
        print('Please provide a file with csv or dataframe format!')

    print('CSV first and last records: \n', df1.head(1), '\n', df1.tail(1))

    # Create new df with downloaded logs file, example name -> "./logs/4Paddle_2024-08-07_to_2024-08-16.log"
    df2 = pd.read_csv(new_logs, sep='\t', names=['counts'])
    # Split data by spacing, then into a group of 3 elements (i.e., column 1: 385 column 2: 0 column 3:0 column 4: Wed Oct 11 17:17:30 2023)
    df2 = df2['counts'].str.split(" ", n=3, expand=True)
    # Create df and indicate column names for new columns
    dft2 = pd.DataFrame(df2.values, columns=["counts", "junk1", "junk2", "date"])
    
    # Format date column to date value
    dft2["date"] = pd.to_datetime(dft2['date'], format="%a %b %d %H:%M:%S %Y")
    # Transform date into current timezone
    dft2['date'] = dft2['date'].dt.tz_localize('America/New_York', ambiguous='infer')
    # convert time zone to UTC (Cordinated Universal Time)
    dft2['date'] = dft2['date'].dt.tz_convert('UTC')

    # make date column the index
    dft2 = dft2.set_index('date')

    # convert counts column data to numeric format -> in case strings appear, it won't cause errors in contrast with astype(int)
    dft2["counts"] = dft2["counts"].apply(pd.to_numeric)

    # clean df by removing columns junk1 and junk2
    dft2.drop(dft2.columns[[1, 2]], axis=1, inplace=True)

    # resampling hourly and getting sum due to logs being recorded for every minute
    dft2_hourly = dft2.resample('h').sum()
    
    # Drop first and last hour on log file due to hourly data count giving innacurate total
    # resulting from starting and ending halfway through the hour/incomplete count
    dft2_hourly.drop(dft2_hourly.head(1).index, inplace=True)
    dft2_hourly.drop(dft2_hourly.tail(1).index, inplace=True)
    print('Log file first and last records: \n', dft2_hourly.head(1), '\n', dft2_hourly.tail(1))
    

    # Merge, keeping latest duplicated indexes into file since they are most likely to have counts
    merged_logs = pd.concat([df1, dft2_hourly])
    final_merged_logs = merged_logs[~merged_logs.index.duplicated(keep='last')]
    # Sort form earliest to latest time
    final_merged_logs2 = final_merged_logs.sort_index()

    # Merged logs into a csv file
    return final_merged_logs2

# Monitor data merging and formatting

# 2Paddle merging

In [ ]:
# Merging 2Paddle csvs with logs
csvname='../data/2Paddle/2paddle_merged_01_12.csv'
logs='../data/2Paddle/2024-05-14_08h58m44s_CH1--020mV_CH2--020mV_CH2--020mV_CH2--020mV_EXT-F.dat'
paddle2_merged = merge_csv_with_logs(csvname,logs)

In [ ]:
logs='../data/2Paddle/2Paddle_2024-07-11_to_2024-08-22.log'
paddle2_merged1 = merge_csv_with_logs(paddle2_merged, logs)

In [ ]:
logs='../data/2Paddle/2paddle.log'
paddle2_merged2 = merge_csv_with_logs(paddle2_merged1, logs)

In [ ]:
new_csv='2paddle_2022_08_03_to_2024_08_22.csv'
paddle2_merged2.to_csv(f'../data/2Paddle/{new_csv}')

In [ ]:
paddle2_merged2.to_sql(
    con=engine, name='paddle2', if_exists='replace', index_label='date')
print('Table sent to db successfully')

In [ ]:
# Fixing csv bc I did not add np.nan values prior to developing final function below
csv_path = '../data/2Paddle/2paddle_2022_08_03_to_2024_08_22.csv'
csv = format_csv(csv_path)
date_range = pd.date_range(start=str(csv.head(1).index[0]), end=str(csv.tail(1).index[0]), freq='1h', tz='UTC')
offline_df = pd.DataFrame(index=date_range, columns=csv.columns, data=np.nan)
complete_logs = pd.concat([offline_df, csv])
merged = complete_logs[~complete_logs.index.duplicated(keep='last')]
final_merged_logs = merged.sort_index()

# Send it to db again
final_merged_logs.to_sql(
    con=engine, name='paddle2', if_exists='replace', index_label='date')
print('Table sent to db successfully')

# Make primary key for table via PSYCOPG2
cur = conn.cursor()
cur.execute(f"""ALTER TABLE paddle2 ADD PRIMARY KEY (date)""")
conn.commit()
cur.close()
print('Query for primary key sent successfully')

In [ ]:
csv_path = '../data/2Paddle/2paddle_all_logs.csv'
final_merged_logs.to_csv(csv_path)

# 4Paddle merging

In [ ]:
csvname='../data/4Paddle/4paddle_merged_01_08.csv'
logs='../data/4Paddle/4Paddle_2024-08-07_to_2024-08-22.log'
paddle4_merged = merge_csv_with_logs(csvname,logs)

In [ ]:
logs='../data/4Paddle/4paddle.log'
paddle4_merged1 = merge_csv_with_logs(paddle4_merged, logs)

In [ ]:
new_csv='4paddle_2019_01_17_to_2024_08_22.csv'
paddle4_merged1.to_csv(f'../data/4Paddle/{new_csv}')

In [ ]:
paddle4_merged1.to_sql(
    con=engine, name='paddle4', if_exists='replace', index_label='date')
print('Table sent to db successfully')

In [ ]:
# Fixing csv bc I did not add np.nan values prior to developing final addressing all
csv_path = '../data/4Paddle/4paddle_2019_01_17_to_2024_08_22.csv'
csv = format_csv(csv_path)
date_range = pd.date_range(start=str(csv.head(1).index[0]), end=str(csv.tail(1).index[0]), freq='1h', tz='UTC')
offline_df = pd.DataFrame(index=date_range, columns=csv.columns, data=np.nan)
complete_logs = pd.concat([offline_df, csv])
merged = complete_logs[~complete_logs.index.duplicated(keep='last')]
final_merged_logs = merged.sort_index()

csv_path = '../data/4Paddle/4paddle_all_logs.csv'
final_merged_logs.to_csv(csv_path)

# Send it to db again
final_merged_logs.to_sql(
    con=engine, name='paddle4', if_exists='replace', index_label='date')
print('Table sent to db successfully')

# Make primary key for table via PSYCOPG2
cur = conn.cursor()
cur.execute(f"""ALTER TABLE paddle4 ADD PRIMARY KEY (date)""")
conn.commit()
cur.close()
print('Query for primary key sent successfully')

# MarkV merging

In [ ]:
log_file='../data/MarkV/2020-07-09_09h51m36s_CH1-0100mV_CH2-0100mV_CH2-0100mV_CH2-0100mV_EXT-T.dat'
logs_csv = log_to_df(log_file)

log_file='../data/MarkV/2020-12-23_14h38m11s_CH1-0100mV_CH2-0100mV_CH2-0100mV_CH2-0100mV_EXT-T.dat'
markv_merged = merge_csv_with_logs(logs_csv, log_file)

In [ ]:
log_file='../data/MarkV/2021-03-22_17h25m31s_CH1-0100mV_CH2-0100mV_CH2-0100mV_CH2-0100mV_EXT-T.dat'
markv_merged2 = merge_csv_with_logs(markv_merged, log_file)

In [ ]:
log_file='../data/MarkV/2021-05-25_12h10m31s_CH1-0100mV_CH2-0100mV_CH2-0100mV_CH2-0100mV_EXT-T.dat'
markv_merged3 = merge_csv_with_logs(markv_merged2, log_file)

In [ ]:
log_file='../data/MarkV/2021-07-08_10h00m51s_CH1-0100mV_CH2-0100mV_CH2-0100mV_CH2-0100mV_EXT-T.dat'
markv_merged4 = merge_csv_with_logs(markv_merged3, log_file)

In [ ]:
log_file='../data/MarkV/2021-11-24_11h09m18s_CH1-0100mV_CH2-0100mV_CH2-0100mV_CH2-0100mV_EXT-T.dat'
markv_merged5 = merge_csv_with_logs(markv_merged4, log_file)

In [ ]:
log_file='../data/MarkV/MarkV_2022-08-08_to_2024-08-22.log'
markv_merged6 = merge_csv_with_logs(markv_merged5, log_file)

In [ ]:
new_csv='MarkV_2020_07_09_to_2023_02_18.csv'
markv_merged6.to_csv(f'../data/MarkV/{new_csv}')

In [ ]:
markv_merged6.to_sql(
    con=engine, name='markv', if_exists='replace', index_label='date')
print('Table sent to db successfully')

In [ ]:
# Fixing csv bc I did not add np.nan values prior to developing final addressing all
csv_path = '../data/MarkV/MarkV_2020_07_09_to_2023_02_18.csv'
csv = format_csv(csv_path)
date_range = pd.date_range(start=str(csv.head(1).index[0]), end=str(csv.tail(1).index[0]), freq='1h', tz='UTC')
offline_df = pd.DataFrame(index=date_range, columns=csv.columns, data=np.nan)
complete_logs = pd.concat([offline_df, csv])
merged = complete_logs[~complete_logs.index.duplicated(keep='last')]
final_merged_logs = merged.sort_index()

csv_path = '../data/MarkV/MarkV_all_logs.csv'
final_merged_logs.to_csv(csv_path)

# Send it to db again
final_merged_logs.to_sql(
    con=engine, name='markv', if_exists='replace', index_label='date')
print('Table sent to db successfully')

# Make primary key for table via PSYCOPG2
cur = conn.cursor()
cur.execute(f"""ALTER TABLE markv ADD PRIMARY KEY (date)""")
conn.commit()
cur.close()
print('Query for primary key sent successfully')

# Abuja Merging

In [ ]:
process_and_save_logs('Abuja')

# APO Merging

In [191]:
process_and_save_logs('APO')

Processing any logs for monitor APO
First date for formatting:  Mon Feb 26 14:08:01 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_57051/3499167562.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_57051/3499167562.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_57051/3499167562.py:50: SettingWithC

First date for formatting:  Mon Feb 26 14:08:01 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_57051/3499167562.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_57051/3499167562.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_57051/3499167562.py:50: SettingWithC

First date for formatting:  Mon Feb 26 14:08:01 2024


...

First date for formatting:  Mon Feb 26 14:08:01 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_57051/3499167562.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_57051/3499167562.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_57051/3499167562.py:50: SettingWithC

Log file index 1
Last log date bf shutdown:  2024-08-22 02:00:00+00:00 
First log date after shutdown:  2024-02-26 22:00:00+00:00
Log file index 2
Last log date bf shutdown:  2024-08-22 02:00:00+00:00 
First log date after shutdown:  2024-02-26 22:00:00+00:00
Log file index 3
Last log date bf shutdown:  2024-08-22 02:00:00+00:00 
First log date after shutdown:  2024-02-26 22:00:00+00:00
Log file index 4
Last log date bf shutdown:  2024-08-22 02:00:00+00:00 
First log date after shutdown:  2024-02-26 22:00:00+00:00


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_57051/3977620479.py:41: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  date_range = pd.date_range(start=last_log_date, end=first_log_date, freq=lfreq, tz='UTC')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_57051/3977620479.py:41: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  date_range = pd.date_range(start=last_log_date, end=first_log_date, freq=lfreq, tz='UTC')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_57051/3977620479.py:41: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  date_range = pd.date_range(start=last_log_date, end=first_log_date, freq=lfreq, tz='UTC')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_57051/3977620479.py:41: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' i

Table sent to DB successfully
Query for primary key sent successfully


# Chara Merging

In [28]:
process_and_save_logs('Chara_Muon002')

Processing any logs for monitor Chara_Muon002
First date for formatting:  2024-07-21T14:11:28.463Z
DF date already timezone aware so no need!


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_92211/3146678621.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_92211/3146678621.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_convert('UTC')


First date for formatting:  2024-07-21T14:11:28.463Z
DF date already timezone aware so no need!


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_92211/3146678621.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_92211/3146678621.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_convert('UTC')


First date for formatting:  2024-07-21T14:11:28.463Z
DF date already timezone aware so no need!


...

First date for formatting:  2022-05-15T17:37:22.512Z
DF date already timezone aware so no need!


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_92211/3146678621.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_92211/3146678621.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_convert('UTC')


First date for formatting:  2023-08-24T18:45:16.365Z
DF date already timezone aware so no need!


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_92211/3146678621.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_92211/3146678621.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_convert('UTC')


Log file index 1
Last log date bf shutdown:  2022-05-15 09:00:00+00:00 
First log date after shutdown:  2022-05-15 18:00:00+00:00
Log file index 2
Last log date bf shutdown:  2022-05-26 11:00:00+00:00 
First log date after shutdown:  2022-06-24 03:00:00+00:00
Log file index 3
Last log date bf shutdown:  2023-02-25 06:00:00+00:00 
First log date after shutdown:  2022-06-24 03:00:00+00:00
Log file index 4
Last log date bf shutdown:  2023-02-25 06:00:00+00:00 
First log date after shutdown:  2023-02-27 03:00:00+00:00
Log file index 5
Last log date bf shutdown:  2023-07-10 14:00:00+00:00 
First log date after shutdown:  2023-08-24 19:00:00+00:00
Log file index 6
Last log date bf shutdown:  2024-01-04 22:00:00+00:00 
First log date after shutdown:  2024-01-05 00:00:00+00:00
Log file index 7
Last log date bf shutdown:  2024-02-23 14:00:00+00:00 
First log date after shutdown:  2024-02-23 16:00:00+00:00
Log file index 8
Last log date bf shutdown:  2024-07-21 13:00:00+00:00 
First log date aft

# ColomboV1 Merging

In [40]:
process_and_save_logs('Colombo/V1')

Processing any logs for monitor Colombo/V1
First date for formatting:  2024-06-05T10:09:40.580Z
DF date already timezone aware so no need!


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_92211/3146678621.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_92211/3146678621.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_convert('UTC')


First date for formatting:  2023-09-20T03:50:36.960Z
DF date already timezone aware so no need!


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_92211/3146678621.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_92211/3146678621.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_convert('UTC')


First date for formatting:  2024-03-23T11:47:47.330Z
DF date already timezone aware so no need!


...

First date for formatting:  2024-06-05T10:09:40.580Z
DF date already timezone aware so no need!


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_92211/3146678621.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_92211/3146678621.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_convert('UTC')


First date for formatting:  2024-06-05T10:09:40.580Z
DF date already timezone aware so no need!


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_92211/3146678621.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_92211/3146678621.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_convert('UTC')


Log file index 1
Last log date bf shutdown:  2023-09-19 18:00:00+00:00 
First log date after shutdown:  2023-09-20 04:00:00+00:00
Log file index 2
Last log date bf shutdown:  2023-12-09 15:00:00+00:00 
First log date after shutdown:  2023-12-10 04:00:00+00:00
Log file index 3
Last log date bf shutdown:  2024-01-08 13:00:00+00:00 
First log date after shutdown:  2024-01-08 16:00:00+00:00
Log file index 4
Last log date bf shutdown:  2024-03-21 20:00:00+00:00 
First log date after shutdown:  2024-03-23 12:00:00+00:00
Log file index 5
Last log date bf shutdown:  2024-03-23 20:00:00+00:00 
First log date after shutdown:  2024-03-24 02:00:00+00:00
Log file index 6
Last log date bf shutdown:  2024-04-04 13:00:00+00:00 
First log date after shutdown:  2024-04-08 14:00:00+00:00
Log file index 7
Last log date bf shutdown:  2024-04-10 20:00:00+00:00 
First log date after shutdown:  2024-04-18 04:00:00+00:00
Log file index 8
Last log date bf shutdown:  2024-06-04 20:00:00+00:00 
First log date aft

# ColomboV2 Merging

In [12]:
process_and_save_logs('Colombo/V2')

Processing any logs for monitor Colombo/V2
First date for formatting:  Tue Aug 20 22:58:28 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94221/3146678621.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94221/3146678621.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94221/3146678621.py:53: SettingWithC

First date for formatting:  Tue Aug 20 22:58:28 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94221/3146678621.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94221/3146678621.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94221/3146678621.py:53: SettingWithC

First date for formatting:  Wed Jul 17 17:24:36 2024


...

First date for formatting:  Wed Jul 17 17:24:36 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94221/3146678621.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94221/3146678621.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94221/3146678621.py:53: SettingWithC

First date for formatting:  Wed Jul 17 17:24:36 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94221/3146678621.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94221/3146678621.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94221/3146678621.py:53: SettingWithC

Log file index 1
Last log date bf shutdown:  2024-08-12 12:00:00+00:00 
First log date after shutdown:  2024-07-17 12:00:00+00:00
Log file index 2
Last log date bf shutdown:  2024-08-12 12:00:00+00:00 
First log date after shutdown:  2024-07-17 12:00:00+00:00
Log file index 3
Last log date bf shutdown:  2024-08-12 12:00:00+00:00 
First log date after shutdown:  2024-08-20 18:00:00+00:00
Log file index 4
Last log date bf shutdown:  2024-08-22 17:00:00+00:00 
First log date after shutdown:  2024-08-20 18:00:00+00:00
Log file index 5
Last log date bf shutdown:  2024-08-22 17:00:00+00:00 
First log date after shutdown:  2024-08-20 18:00:00+00:00
Table sent to DB successfully
Query for primary key sent successfully


# One Park Place Merging

In [12]:
process_and_save_logs('OneParkPlace')

Processing any logs for monitor OneParkPlace
First date for formatting:  Mon Jul 15 15:44:04 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:53: SettingWithC

First date for formatting:  Mon Jul 15 15:44:04 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:53: SettingWithC

First date for formatting:  Mon Jul 15 15:44:04 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:53: SettingWithC

First date for formatting:  Mon Jul 15 15:44:04 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:53: SettingWithC

Log file index 1
Last log date bf shutdown:  2024-08-21 03:00:00+00:00 
First log date after shutdown:  2024-07-15 20:00:00+00:00
Log file index 2
Last log date bf shutdown:  2024-08-21 03:00:00+00:00 
First log date after shutdown:  2024-07-15 20:00:00+00:00
Log file index 3
Last log date bf shutdown:  2024-08-21 03:00:00+00:00 
First log date after shutdown:  2024-07-15 20:00:00+00:00
Table sent to DB successfully
Query for primary key sent successfully


# Rm415_Muon001 merging

In [13]:
process_and_save_logs('Rm415_Muon001')

Processing any logs for monitor Rm415_Muon001
First date for formatting:  Wed May  8 18:18:30 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:53: SettingWithC

First date for formatting:  Wed May  8 18:18:30 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:53: SettingWithC

First date for formatting:  Wed May  8 18:18:30 2024


...

First date for formatting:  Fri Aug 23 15:54:28 2024
First date for formatting:  Fri Aug 23 15:54:28 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:53: SettingWithC

First date for formatting:  Sun Mar 24 20:40:08 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3146678621.py:53: SettingWithC

Log file index 1
Last log date bf shutdown:  2024-05-03 09:00:00+00:00 
First log date after shutdown:  2024-05-03 12:00:00+00:00
Log file index 2
Last log date bf shutdown:  2024-05-08 20:00:00+00:00 
First log date after shutdown:  2024-05-08 23:00:00+00:00
Log file index 3
Last log date bf shutdown:  2024-08-21 04:00:00+00:00 
First log date after shutdown:  2024-05-08 23:00:00+00:00
Log file index 4
Last log date bf shutdown:  2024-08-21 04:00:00+00:00 
First log date after shutdown:  2024-05-08 23:00:00+00:00
Log file index 5
Last log date bf shutdown:  2024-08-21 04:00:00+00:00 
First log date after shutdown:  2024-05-08 23:00:00+00:00
Log file index 6
Last log date bf shutdown:  2024-08-21 04:00:00+00:00 
First log date after shutdown:  2024-08-23 20:00:00+00:00
Log file index 7
Last log date bf shutdown:  2024-08-25 02:00:00+00:00 
First log date after shutdown:  2024-08-23 20:00:00+00:00
Table sent to DB successfully
Query for primary key sent successfully


# SantaMarta merging

In [81]:
process_and_save_logs('SantaMarta')

Processing any logs for monitor SantaMarta
     0   1    2                         3
0  135  69  160  Sat Apr 27 14:02:09 2024
First date for formatting:  Sat Apr 27 14:02:09 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_1055/4005775305.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_1055/4005775305.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_1055/4005775305.py:61: SettingWithCopy

date check prior to localize:                   date counts
0 2024-04-27 14:02:09    135
     0   1    2                         3
0  143  70  165  Fri Apr 26 11:36:00 2024
First date for formatting:  Fri Apr 26 11:36:00 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_1055/4005775305.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_1055/4005775305.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_1055/4005775305.py:61: SettingWithCopy

date check prior to localize:                   date counts
0 2024-04-26 11:36:00    143
     0   1    2                         3
0  149  58  102  Sat Feb 24 21:18:43 2024
First date for formatting:  Sat Feb 24 21:18:43 2024
date check prior to localize:                   date counts
0 2024-02-24 21:18:43    149
     0   1    2                         3
0  102  53  105  Mon Aug 14 14:30:23 2023
First date for formatting:  Mon Aug 14 14:30:23 2023


...

date check prior to localize:                   date counts
0 2024-02-25 11:33:36    169
    0   1    2                         3
0  89  45  114  Sun May 19 18:48:48 2024
First date for formatting:  Sun May 19 18:48:48 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_1055/4005775305.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_1055/4005775305.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_1055/4005775305.py:61: SettingWithCopy

date check prior to localize:                   date counts
0 2024-05-19 18:48:48     89
     0   1    2                         3
0  149  68  117  Sat Feb 24 10:18:42 2024
First date for formatting:  Sat Feb 24 10:18:42 2024
date check prior to localize:                   date counts
0 2024-02-24 10:18:42    149
Log file index 1
Last log date bf shutdown:  2023-08-29 18:00:00+00:00 
First log date after shutdown:  2023-08-15 23:00:00+00:00
Log file index 2
Last log date bf shutdown:  2023-08-29 18:00:00+00:00 
First log date after shutdown:  2023-08-15 23:00:00+00:00
Log file index 3
Last log date bf shutdown:  2023-08-29 18:00:00+00:00 
First log date after shutdown:  2023-08-15 23:00:00+00:00
Log file index 4
Last log date bf shutdown:  2023-08-29 18:00:00+00:00 
First log date after shutdown:  2023-08-15 23:00:00+00:00
Log file index 5
Last log date bf shutdown:  2023-08-29 18:00:00+00:00 
First log date after shutdown:  2024-02-06 22:00:00+00:00
Log file index 6
Last log date bf s

/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_1055/4005775305.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_1055/4005775305.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_1055/4005775305.py:61: SettingWithCopy

Log file index 17
Last log date bf shutdown:  2024-05-19 11:00:00+00:00 
First log date after shutdown:  2024-05-20 05:00:00+00:00
Log file index 18
Last log date bf shutdown:  2024-08-01 11:00:00+00:00 
First log date after shutdown:  2024-08-06 23:00:00+00:00
Log file index 19
Last log date bf shutdown:  2024-08-09 12:00:00+00:00 
First log date after shutdown:  2024-08-13 22:00:00+00:00
Log file index 20
Last log date bf shutdown:  2024-08-20 18:00:00+00:00 
First log date after shutdown:  2024-08-13 22:00:00+00:00
Log file index 21
Last log date bf shutdown:  2024-08-20 18:00:00+00:00 
First log date after shutdown:  2024-08-13 22:00:00+00:00
Log file index 22
Log file index 23
Log file index 24
Table sent to DB successfully
Query for primary key sent successfully


# Serbia_Belgrade Det1 Merging

In [46]:
process_and_save_logs('Serbia_Belgrade/Det1')

Processing any logs for monitor Serbia_Belgrade/Det1
      0    1    2    3                         4
0  1505  217  107  204  Tue Jun 18 11:24:09 2024
First date for formatting:  Tue Jun 18 11:24:09 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:54: SettingWithC

                 date counts
0 2024-06-18 11:24:09    217
      0    1    2    3                         4
0  2542  218  117  234  Mon Jul 22 15:06:18 2024
First date for formatting:  Mon Jul 22 15:06:18 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:54: SettingWithC

                 date counts
0 2024-07-22 15:06:18    218
      0    1    2    3                         4
0  2542  218  117  234  Mon Jul 22 15:06:18 2024
First date for formatting:  Mon Jul 22 15:06:18 2024


...

                 date counts
0 2024-07-22 15:06:18    218
      0    1    2    3                         4
0  2542  218  117  234  Mon Jul 22 15:06:18 2024
First date for formatting:  Mon Jul 22 15:06:18 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:54: SettingWithC

                 date counts
0 2024-07-22 15:06:18    218
      0    1   2    3                         4
0  1510  180  89  208  Tue Jul  9 00:26:47 2024
First date for formatting:  Tue Jul  9 00:26:47 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:54: SettingWithC

                 date counts
0 2024-07-09 00:26:47    180
Log file index 1
Last log date bf shutdown:  2024-07-08 19:00:00+00:00 
First log date after shutdown:  2024-07-08 23:00:00+00:00
Log file index 2
Last log date bf shutdown:  2024-07-22 10:00:00+00:00 
First log date after shutdown:  2024-07-22 14:00:00+00:00
Log file index 3
Last log date bf shutdown:  2024-08-24 02:00:00+00:00 
First log date after shutdown:  2024-07-22 14:00:00+00:00
Log file index 4
Last log date bf shutdown:  2024-08-24 02:00:00+00:00 
First log date after shutdown:  2024-07-22 14:00:00+00:00
Log file index 5
Last log date bf shutdown:  2024-08-24 02:00:00+00:00 
First log date after shutdown:  2024-07-22 14:00:00+00:00
Log file index 6
Last log date bf shutdown:  2024-08-24 02:00:00+00:00 
First log date after shutdown:  2024-07-22 14:00:00+00:00
Log file index 7
Last log date bf shutdown:  2024-08-24 02:00:00+00:00 
First log date after shutdown:  2024-07-22 14:00:00+00:00
Table sent to DB successfully
Qu

# Serbia_Belgrade Det2 Merging

In [47]:
process_and_save_logs('Serbia_Belgrade/Det2')

Processing any logs for monitor Serbia_Belgrade/Det2
      0    1   2    3                         4
0  1641  197  94  201  Tue Jul  9 00:26:47 2024
First date for formatting:  Tue Jul  9 00:26:47 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:54: SettingWithC

                 date counts
0 2024-07-09 00:26:47    197
      0    1    2    3                         4
0  2740  214  117  242  Mon Jul 22 15:06:18 2024
First date for formatting:  Mon Jul 22 15:06:18 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:54: SettingWithC

                 date counts
0 2024-07-22 15:06:18    214
      0    1    2    3                         4
0  2740  214  117  242  Mon Jul 22 15:06:18 2024
First date for formatting:  Mon Jul 22 15:06:18 2024


...

                 date counts
0 2024-07-22 15:06:18    214
      0    1    2    3                         4
0  2740  214  117  242  Mon Jul 22 15:06:18 2024
First date for formatting:  Mon Jul 22 15:06:18 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:54: SettingWithC

                 date counts
0 2024-07-22 15:06:18    214
      0    1    2    3                         4
0  1604  197  105  222  Mon Jun 17 13:20:59 2024
First date for formatting:  Mon Jun 17 13:20:59 2024


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_localize(monitor_tz, ambiguous='infer', nonexistent='shift_forward')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_94875/3466536323.py:54: SettingWithC

                 date counts
0 2024-06-17 13:20:59    197
Log file index 1
Last log date bf shutdown:  2024-07-08 19:00:00+00:00 
First log date after shutdown:  2024-07-08 23:00:00+00:00
Log file index 2
Last log date bf shutdown:  2024-07-22 10:00:00+00:00 
First log date after shutdown:  2024-07-22 14:00:00+00:00
Log file index 3
Last log date bf shutdown:  2024-08-19 08:00:00+00:00 
First log date after shutdown:  2024-07-22 14:00:00+00:00
Log file index 4
Last log date bf shutdown:  2024-08-19 08:00:00+00:00 
First log date after shutdown:  2024-07-22 14:00:00+00:00
Log file index 5
Last log date bf shutdown:  2024-08-19 08:00:00+00:00 
First log date after shutdown:  2024-07-22 14:00:00+00:00
Log file index 6
Last log date bf shutdown:  2024-08-19 08:00:00+00:00 
First log date after shutdown:  2024-07-22 14:00:00+00:00
Log file index 7
Last log date bf shutdown:  2024-08-19 08:00:00+00:00 
First log date after shutdown:  2024-07-22 14:00:00+00:00
Table sent to DB successfully
Qu

# UvaWellassa merging

In [12]:
process_and_save_logs('UvaWellassa_Muon001')

Processing any logs for monitor UvaWellassa_Muon001
                         0  1  2  3   4   5   6  7    8    9    10 11
1  2024-07-21T05:28:11.699Z  0  0  0  61  25  51  0  522  421  457   
First date for formatting:  2024-07-21T05:28:11.699Z


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_1055/3466536323.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_1055/3466536323.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_convert('UTC')


                              date counts
1 2024-07-21 05:28:11.699000+00:00     61
DF date already timezone aware so no need!
                         0  1  2  3   4   5   6  7    8    9    10 11
1  2024-07-21T05:28:11.699Z  0  0  0  61  25  51  0  522  421  457   
First date for formatting:  2024-07-21T05:28:11.699Z
                              date counts
1 2024-07-21 05:28:11.699000+00:00     61
DF date already timezone aware so no need!


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_1055/3466536323.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_1055/3466536323.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_convert('UTC')


                         0  1  2  3   4   5   6  7    8    9    10 11
1  2024-07-21T05:28:11.699Z  0  0  0  61  25  51  0  522  421  457   
First date for formatting:  2024-07-21T05:28:11.699Z
                              date counts
1 2024-07-21 05:28:11.699000+00:00     61
DF date already timezone aware so no need!


...

                         0  1  2  3   4   5   6  7    8    9    10 11
1  2024-05-29T13:33:47.555Z  0  0  0  99  37  94  0  639  540  542   
First date for formatting:  2024-05-29T13:33:47.555Z
                              date counts
1 2024-05-29 13:33:47.555000+00:00     99
DF date already timezone aware so no need!


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_1055/3466536323.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_1055/3466536323.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_convert('UTC')


                         0  1  2  3   4   5   6  7    8    9    10 11
1  2024-07-21T05:28:11.699Z  0  0  0  61  25  51  0  522  421  457   
First date for formatting:  2024-07-21T05:28:11.699Z
                              date counts
1 2024-07-21 05:28:11.699000+00:00     61
DF date already timezone aware so no need!


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_1055/3466536323.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = pd.to_datetime(ldf['date'], format='mixed')
/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_1055/3466536323.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['date'] = ldf['date'].dt.tz_convert('UTC')


Log file index 1
Last log date bf shutdown:  2024-05-29 01:00:00+00:00 
First log date after shutdown:  2024-05-29 14:00:00+00:00
Log file index 2
Last log date bf shutdown:  2024-07-18 01:00:00+00:00 
First log date after shutdown:  2024-07-18 13:00:00+00:00
Log file index 3
Last log date bf shutdown:  2024-07-21 01:00:00+00:00 
First log date after shutdown:  2024-07-21 06:00:00+00:00
Log file index 4
Last log date bf shutdown:  2024-08-21 01:00:00+00:00 
First log date after shutdown:  2024-07-21 06:00:00+00:00
Log file index 5
Last log date bf shutdown:  2024-08-21 01:00:00+00:00 
First log date after shutdown:  2024-07-21 06:00:00+00:00
Log file index 6
Last log date bf shutdown:  2024-08-21 01:00:00+00:00 
First log date after shutdown:  2024-07-21 06:00:00+00:00
Log file index 7
Last log date bf shutdown:  2024-08-21 01:00:00+00:00 
First log date after shutdown:  2024-07-21 06:00:00+00:00
Log file index 8
Last log date bf shutdown:  2024-08-21 01:00:00+00:00 
First log date aft

In [93]:
# Temporary function to fix any 0 vals on dfs due to resample function
homedir = '../data/'
a = listdir(homedir)
detectors = [l for l in a if l != 'unsorted' and '.' not in l]

for d in detectors:
    b = listdir(homedir+d)
    sublist = [l for l in b if l != '.DS_Store' and 'all_logs.csv' in l]
    if len(sublist) == 0:
        sublist = [l for l in b if l != '.DS_Store' and '.' not in l]
        for sub in sublist:
            c = listdir(homedir+d+'/'+sub)
            subsublist = [l for l in c if l != '.DS_Store' and 'all_logs.csv' in l]

            df = format_csv(homedir+d+'/'+sub+'/'+subsublist[0])
            df.loc[df['counts'] == 0] = np.nan
            df.to_csv(homedir+d+'/'+sub+'/'+subsublist[0])

            monitor_name = subsublist[0][:subsublist[0].find('_all')].lower()

            if monitor_name.startswith('2') or monitor_name.startswith('4'):
                monitor_name = monitor_name[1:]+monitor_name[0]

            df.to_sql(
                con=engine, 
                name=f'{monitor_name}', 
                if_exists='replace', 
                index_label='date'
            )
            print('Table sent to DB successfully')

            # Make primary key for table via PSYCOPG2
            cur = conn.cursor()
            cur.execute(f"""ALTER TABLE {monitor_name} ADD PRIMARY KEY (date)""")
            conn.commit()
    else:
        df = format_csv(homedir+d+'/'+sublist[0])
        df.loc[df['counts'] == 0] = np.nan
        df.to_csv(homedir+d+'/'+sublist[0])

        monitor_name = sublist[0][:sublist[0].find('_all')].lower()

        if monitor_name.startswith('2') or monitor_name.startswith('4'):
            monitor_name = monitor_name[1:]+monitor_name[0]

        df.to_sql(
            con=engine, 
            name=f'{monitor_name}', 
            if_exists='replace', 
            index_label='date'
        )
        print('Table sent to DB successfully')

        # Make primary key for table via PSYCOPG2
        cur = conn.cursor()
        cur.execute(f"""ALTER TABLE {monitor_name} ADD PRIMARY KEY (date)""")
        conn.commit()


Table sent to DB successfully
Table sent to DB successfully
Table sent to DB successfully
Table sent to DB successfully
Table sent to DB successfully
Table sent to DB successfully
Table sent to DB successfully
Table sent to DB successfully
Table sent to DB successfully


/var/folders/tg/f26try5n4pq7thttks3b9fs00000gn/T/ipykernel_1055/3691705342.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(csv_path, names=['date','counts'])[1:] # Remove extra first row


Table sent to DB successfully
Table sent to DB successfully
Table sent to DB successfully
Table sent to DB successfully
Table sent to DB successfully
